In [5]:
# file dependencies
from tacc_stats.pickler.job_stats import Job
import cPickle as pickle
import argparse
from os import listdir

In [6]:
# data manipulation dependencies
import pandas as pd
import numpy as np
import datetime as dt

In [7]:
# pickle file with job data
pickle_file = open('/oasis/projects/nsf/sys200/tcooper/xsede_stats/comet_pickles/2016-08-21/3864417', 'rb')

In [8]:
# open pickle file
# extract job
jobid = pickle.load(pickle_file)
pickle_file.close()

In [9]:
# general stats
hosts = jobid.hosts
start = pd.to_datetime(round(jobid.start_time), unit='s').time()
end = pd.to_datetime(round(jobid.end_time), unit='s').time()
numCycles = len(jobid.times) + 2
stat_types = jobid.hosts['comet-03-09'].stats.keys()

In [10]:
type_avgs = {}
times = [start]
devices = []

for host_name, host in jobid.hosts.iteritems():
    typ='all'
    dev='all'
    
    if 'all' in typ: typ_keys = host.stats.keys()
    else: typ_keys = typ
        
    for time in host.times:
        times.append( pd.to_datetime(round(time), unit='s').time() )
    
    for type_name in typ_keys:
        type_data = host.stats[type_name]
        type_avgs[type_name] = []
        
        if 'all' in dev: dev_keys = type_data.keys()
        else: dev_keys = dev
            
        for dev_name in dev_keys:          
            devices.append( (dev_name,type_name) )
            type_avgs[type_name].append( [np.mean(row).round(decimals=3)
                                   for row in type_data[dev_name]] )
times.append(end)

In [11]:
devices

[('wombat-clilov', 'mdc'),
 ('panda-clilov', 'mdc'),
 ('-', 'vm'),
 ('0/09.2', 'intel_hsw_qpi'),
 ('1/08.2', 'intel_hsw_qpi'),
 ('0/08.2', 'intel_hsw_qpi'),
 ('1/09.2', 'intel_hsw_qpi'),
 ('1/1', 'intel_hsw_cbo'),
 ('1/0', 'intel_hsw_cbo'),
 ('1/11', 'intel_hsw_cbo'),
 ('1/10', 'intel_hsw_cbo'),
 ('1/7', 'intel_hsw_cbo'),
 ('1/6', 'intel_hsw_cbo'),
 ('1/5', 'intel_hsw_cbo'),
 ('1/4', 'intel_hsw_cbo'),
 ('1/3', 'intel_hsw_cbo'),
 ('1/2', 'intel_hsw_cbo'),
 ('0/8', 'intel_hsw_cbo'),
 ('0/9', 'intel_hsw_cbo'),
 ('0/6', 'intel_hsw_cbo'),
 ('0/7', 'intel_hsw_cbo'),
 ('0/4', 'intel_hsw_cbo'),
 ('0/5', 'intel_hsw_cbo'),
 ('0/2', 'intel_hsw_cbo'),
 ('0/3', 'intel_hsw_cbo'),
 ('0/0', 'intel_hsw_cbo'),
 ('0/1', 'intel_hsw_cbo'),
 ('0/10', 'intel_hsw_cbo'),
 ('0/11', 'intel_hsw_cbo'),
 ('1/9', 'intel_hsw_cbo'),
 ('1/8', 'intel_hsw_cbo'),
 ('1', 'intel_hsw_pcu'),
 ('0', 'intel_hsw_pcu'),
 ('wombat-clilov', 'osc'),
 ('panda-clilov', 'osc'),
 ('mlx4_0.1', 'ib'),
 ('20', 'intel_hsw'),
 ('21', 'intel_

In [12]:
all_stats = { time:{ stat:[] for stat in type_avgs } for time in times }

In [13]:
for type_avg,data in type_avgs.iteritems():
    for dev in range( len(data) ):
        all_stats[dt.time(21, 59, 44)][type_avg] .append(  data[dev][0]  .round())
        all_stats[dt.time(22, 9, 45)] [type_avg] .append(   data[dev][1] .round())   
        all_stats[dt.time(22, 19, 45)][type_avg] .append(  data[dev][2]  .round()) 
        all_stats[dt.time(22, 29, 45)][type_avg] .append(  data[dev][3]  .round())
        all_stats[dt.time(22, 39, 45)][type_avg] .append(  data[dev][4]  .round())
        all_stats[dt.time(22, 49, 45)][type_avg] .append(  data[dev][5]  .round())   
        all_stats[dt.time(0, 38, 3)]  [type_avg] .append(    data[dev][6].round())

### Descriptive Statistics Across Devices

#### Observing the general pattern of variation between devices for each type of statistic, in support of detecting accuracy of averages used

In [14]:
dev_stats = { type_stat:{} for type_stat in stat_types}

In [15]:
for time,stat in all_stats.iteritems():
    for stat,avgList in stat.iteritems():
        dev_stats[stat]["Average by Device"] = avgList
        dev_stats[stat]["Overall Average"] = np.mean(avgList)
        dev_stats[stat]["Standard Deviation"] = np.std(avgList)
        dev_stats[stat]["High"] = np.max(avgList)
        dev_stats[stat]["Low"] = np.min(avgList)
        
        all_stats[time][stat] = np.mean(avgList)

In [16]:
desc_df = pd.DataFrame( dev_stats ).drop('Average by Device', axis=0 )
desc_df

,block,cpu,ib,ib_ext,ib_sw,intel_hsw,intel_hsw_cbo,intel_hsw_hau,intel_hsw_imc,intel_hsw_pcu,...,net,nfs,numa,osc,proc,ps,sysv_shm,tmpfs,vfs,vm
High,2.08676e+07,42844,0,5.59724e+09,1.39875e+10,2.71434e+12,8.53007e+11,2.53178e+12,7.58326e+11,7.54966e+11,...,1.67843e+09,300529,1.72527e+07,62443,7.3787e+18,6.56631e+06,0,2,403900,3.52013e+07
Low,0,42435,0,5.59724e+09,1.39875e+10,2.16492e+09,1.45602e+11,2.25263e+12,6.40926e+11,7.03824e+11,...,1130,0,7.29715e+06,55451,33608,6.56631e+06,0,2,403900,3.52013e+07
Overall Average,1.09625e+07,42791.1,0,5.59724e+09,1.39875e+10,1.27788e+12,4.30336e+11,2.39221e+12,6.99462e+11,7.29395e+11,...,4.79776e+08,88660.8,1.22749e+07,58947,6.62191e+17,6.56631e+06,0,2,403900,3.52013e+07
Standard Deviation,9.91751e+06,84.5133,0,0,0,9.2481e+11,2.82124e+11,1.39575e+11,5.85019e+10,2.55712e+10,...,6.61004e+08,124227,4.97779e+06,3496,1.9662e+18,0,0,0,0,0


### Focus on Trends

#### Continuation of developing dataframe

In [17]:
all_stats

{datetime.time(0, 38, 3): {'block': 11241274.25,
  'cpu': 43924.708333333336,
  'ib': 0.0,
  'ib_ext': 5635720866.0,
  'ib_sw': 14083693436.0,
  'intel_hsw': 1306175137326.2917,
  'intel_hsw_cbo': 441898973602.625,
  'intel_hsw_hau': 2455565252230.5,
  'intel_hsw_imc': 717965801725.125,
  'intel_hsw_pcu': 738496093222.5,
  'intel_hsw_qpi': 330643396049.75,
  'intel_hsw_r2pci': 1085601473503.5,
  'llite': 1165814886.0,
  'lnet': 6372874127.0,
  'mdc': 54376.0,
  'mem': 15753095168.0,
  'net': 483060553.39999998,
  'nfs': 93288.5,
  'numa': 12567728.5,
  'osc': 61535.0,
  'proc': 1.2297829382476877e+18,
  'ps': 6739856.0,
  'sysv_shm': 0.0,
  'tmpfs': 2.0,
  'vfs': 404499.0,
  'vm': 36115552.0},
 datetime.time(21, 59, 44): {'block': 0.0,
  'cpu': 0.0,
  'ib': 0.0,
  'ib_ext': 0.0,
  'ib_sw': 0.0,
  'intel_hsw': 0.0,
  'intel_hsw_cbo': 0.0,
  'intel_hsw_hau': 0.0,
  'intel_hsw_imc': 0.0,
  'intel_hsw_pcu': 0.0,
  'intel_hsw_qpi': 0.0,
  'intel_hsw_r2pci': 0.0,
  'llite': 0.0,
  'lnet': 0.

In [18]:
# set indices
indexList = ["First", "Second", "Third", "Fourth", "Fifth", "Sixth", "Seventh"]

# index by time of check
df_1 = pd.DataFrame( all_stats[dt.time(21, 59, 44)], index=["First"] )#.set_index(indexList)
df_2 = pd.DataFrame( all_stats[dt.time(22, 9, 45)],  index=["Second"] ) #.set_index(indexList)
df_3 = pd.DataFrame( all_stats[dt.time(22, 19, 45)], index=["Third"] )#.set_index(indexList)
df_4 = pd.DataFrame( all_stats[dt.time(22, 29, 45)], index=["Fourth"] )#.set_index(indexList)
df_5 = pd.DataFrame( all_stats[dt.time(22, 39, 45)], index=["Fifth"] )#.set_index(indexList)
df_6 = pd.DataFrame( all_stats[dt.time(22, 49, 45)], index=["Sixth"] )#.set_index(indexList)
df_7 = pd.DataFrame( all_stats[dt.time(0, 38, 3)],   index=["Seventh"] )  #.set_index(indexList)

# merge dataframes
df_time = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7])
df_time.index.name = "Cycle"
df_time

,block,cpu,ib,ib_ext,ib_sw,intel_hsw,intel_hsw_cbo,intel_hsw_hau,intel_hsw_imc,intel_hsw_pcu,...,net,nfs,numa,osc,proc,ps,sysv_shm,tmpfs,vfs,vm
Cycle,,,,,,,,,,,,,,,,,,,,,
First,0.00,0.000000,0,0,0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.00,0.0,0.0,3.721382e+05,399,0,2,415154,0
Second,2210393.25,8560.125000,0,742707505,1855977587,2.184649e+11,8.337045e+10,4.779738e+11,1.395632e+11,1.697021e+11,...,63618267.8,17621.00,1938144.0,12965.5,5.675921e+17,1392432,0,2,415400,7273413
Third,4363898.50,17116.625000,0,2124022459,5307860778,5.001884e+11,1.680856e+11,9.564698e+11,2.796058e+11,3.063754e+11,...,182035070.8,36200.50,4001166.0,23145.5,5.675921e+17,2661077,0,2,415301,13441317
Fourth,6568380.50,25673.416667,0,3146897070,7863993849,7.418263e+11,2.540592e+11,1.434754e+12,4.193947e+11,4.590157e+11,...,269732900.4,49999.00,6570716.5,33587.0,5.675921e+17,3963527,0,2,415325,20499563
Fifth,8689852.00,34230.625000,0,4337567698,10839530636,1.046498e+12,3.442219e+11,1.913866e+12,5.597273e+11,5.710779e+11,...,371739373.2,64791.75,9634908.0,46947.5,6.621908e+17,5209279,0,2,410832,27739777
Sixth,10962533.75,42791.083333,0,5597235685,13987514012,1.277884e+12,4.303361e+11,2.392207e+12,6.994621e+11,7.293947e+11,...,479776010.4,88660.75,12274940.0,58947.0,6.621908e+17,6566311,0,2,403900,35201308
Seventh,11241274.25,43924.708333,0,5635720866,14083693436,1.306175e+12,4.418990e+11,2.455565e+12,7.179658e+11,7.384961e+11,...,483060553.4,93288.50,12567728.5,61535.0,1.229783e+18,6739856,0,2,404499,36115552


In [19]:
df_time = df_time[["intel_hsw","ib","llite",
         # "ib_sw","ib_ext", "mdc","osc",
         "lnet", "block","cpu","mem", "net",
         "nfs","numa","proc","sysv_shm","tmpfs"]]

In [20]:
# select and rename stats
df_time = df_time.rename( columns={"index": "Job Name",
                         "amd64_pmc": "AMD Opteron performance counters (per core)",
                        "intel_hsw": "Intel Haswell Processor (HSW) (per core)",
                        "intel_hsw_ht": "Intel Haswell Processor - Hyper-threaded (per logical core)",
                        "intel_nhm": "Intel Nehalem Processor (NHM) (per core)",
                        "intel_uncore": "Westmere Uncore (WTM) (per socket)",
                        "intel_snb": "Intel Sandy Brige (SNB) or Ivy Bridge (IVB) Processor (per core)",
                        "intel_snb(hsw)_cbo": "Caching Agent (CBo) for SNB (HSW) (per socket)",
                        "intel_snb(hsw)_pcu": "Power Control Unit for SNB (HSW) (per socket)",
                        "intel_snb(hsw)_imc": "Integrated Memory Controller for SNB (HSW) (per socket)",
                        "intel_snb(hsw)_qpi": "QPI Link Layer for SNB (HSW) (per socket)",
                        "intel_snb(hsw)_hau": "Home Agent Unit for SNB (HSW) (per socket)",
                        "intel_snb(hsw)_r2pci": "Ring to PCIe Agent for SNB (HSW) (per socket)",
                        "ib": "Infiniband usage (default)",
                        "ib_sw": "InfiniBand usage (sw)",
                        "ib_ext": "Infiniband usage (ext)",
                        "llite": "Lustre filesystem usage (per mount)",
                        "lnet": "Lustre network usage (lnet)",
                        "mdc": "Lustre network usage (mdc)",
                        "mic": "MIC scheduler account (per hardware thread)",
                        "osc": "Lustre filesystem usage (osc)",
                        "block": "Block device statistics (per device)",
                        "cpu": "Scheduler accounting (per CPU)",
                        "mem": "Memory usage (per socket)",
                        "net": "Network device usage (per device)",
                        "nfs": "NFS system usage",
                        "numa": "NUMA statistics (per socket)",
                        "proc": "Process specific data (MaxRSS, executable name etc.)",
                        "ps": "Process statistics",
                        "sysv_shm": "SysV shared memory segment usage",
                        "tmpfs": "Ram-backed filesystem usage (per mount)",
                        "vfs": "Dentry/file/inode cache usage",
                        "vm": "Virtual memory statistics"
                        })

In [21]:
# reorder columns
#df_time = df_time.reindex(columns=[
#                           "Block Device Statistics (per device)",
#                           "Intel Haswell Processor (per core)",
#                           "Infiniband Usage",
#                           #"Infiniband Usage",
#                           #"Infiniband Usage",
#                           "Lustre Filesystem Usage (per mount)",
#                           "Lustre Network Usage",
#                           #"Lustre Network Usage",
#                           #"Lustre Filesystem Usage",
#                           "Memory Usage",
#                           "Network Device Usage (per device)",
#                           "NFS System Usage",
#                           "NUMA Statistics (per socket)",
#                           "Process Specific Data",
#                           "RAM-Backed Filesystem Usage (per mount)",
#                           "Scheduler Accounting (per CPU)",
#                           "System Shared Memory Segment Usage"])

In [22]:
df_time

,Intel Haswell Processor (HSW) (per core),Infiniband usage (default),Lustre filesystem usage (per mount),Lustre network usage (lnet),Block device statistics (per device),Scheduler accounting (per CPU),Memory usage (per socket),Network device usage (per device),NFS system usage,NUMA statistics (per socket),"Process specific data (MaxRSS, executable name etc.)",SysV shared memory segment usage,Ram-backed filesystem usage (per mount)
Cycle,,,,,,,,,,,,,
First,0.000000e+00,0,0.000000e+00,0,0.00,0.000000,13568753664,0.0,0.00,0.0,3.721382e+05,0,2
Second,2.184649e+11,0,1.055711e+08,1160953426,2210393.25,8560.125000,14044895744,63618267.8,17621.00,1938144.0,5.675921e+17,0,2
Third,5.001884e+11,0,3.164712e+08,2382550442,4363898.50,17116.625000,14916957056,182035070.8,36200.50,4001166.0,5.675921e+17,0,2
Fourth,7.418263e+11,0,4.254576e+08,3031516248,6568380.50,25673.416667,15796083840,269732900.4,49999.00,6570716.5,5.675921e+17,0,2
Fifth,1.046498e+12,0,6.408629e+08,4311930214,8689852.00,34230.625000,15956769664,371739373.2,64791.75,9634908.0,6.621908e+17,0,2
Sixth,1.277884e+12,0,1.053455e+09,6310672441,10962533.75,42791.083333,16010846080,479776010.4,88660.75,12274940.0,6.621908e+17,0,2
Seventh,1.306175e+12,0,1.165815e+09,6372874127,11241274.25,43924.708333,15753095168,483060553.4,93288.50,12567728.5,1.229783e+18,0,2


In [24]:
df_time.to_csv(path_or_buf="./test_job.csv")

In [25]:
pd.read_csv("./test_job.csv")

,Cycle,Intel Haswell Processor (HSW) (per core),Infiniband usage (default),Lustre filesystem usage (per mount),Lustre network usage (lnet),Block device statistics (per device),Scheduler accounting (per CPU),Memory usage (per socket),Network device usage (per device),NFS system usage,NUMA statistics (per socket),"Process specific data (MaxRSS, executable name etc.)",SysV shared memory segment usage,Ram-backed filesystem usage (per mount)
0,First,0.000000e+00,0,0.000000e+00,0,0.00,0.000000,13568753664,0.0,0.00,0.0,3.721382e+05,0,2
1,Second,2.184649e+11,0,1.055711e+08,1160953426,2210393.25,8560.125000,14044895744,63618267.8,17621.00,1938144.0,5.675921e+17,0,2
2,Third,5.001884e+11,0,3.164712e+08,2382550442,4363898.50,17116.625000,14916957056,182035070.8,36200.50,4001166.0,5.675921e+17,0,2
3,Fourth,7.418263e+11,0,4.254576e+08,3031516248,6568380.50,25673.416667,15796083840,269732900.4,49999.00,6570716.5,5.675921e+17,0,2
4,Fifth,1.046498e+12,0,6.408629e+08,4311930214,8689852.00,34230.625000,15956769664,371739373.2,64791.75,9634908.0,6.621908e+17,0,2
5,Sixth,1.277884e+12,0,1.053455e+09,6310672441,10962533.75,42791.083333,16010846080,479776010.4,88660.75,12274940.0,6.621908e+17,0,2
6,Seventh,1.306175e+12,0,1.165815e+09,6372874127,11241274.25,43924.708333,15753095168,483060553.4,93288.50,12567728.5,1.229783e+18,0,2
